<a href="https://colab.research.google.com/github/rjhanjee/my-first-repo/blob/main/hw4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
import sqlite3 as sq3
import pandas as pd

# I imported the county_pop_arcos.csv, land_area.csv and county_annual.csv files into opioid.db file
# in sqlite on my desktop as shown by using same steps as shown in the book

## Create the connection
con = sq3.connect("opioid.db")

population = pd.read_sql_query("SELECT * FROM population", con) # data from county_pop_arcos.csv
annual = pd.read_sql_query("SELECT * FROM annual", con) # data from county_annual.csv
land = pd.read_sql_query("SELECT * FROM land", con) # data from land_area.csv

# you have to close the connection
con.close()

print(population[['BUYER_COUNTY', 'BUYER_STATE','STATE','COUNTY','year','population']].head())

# population
# select BUYER_COUNTY, BUYER_STATE, STATE, COUNTY, year, population from population limit


# print(" population data sample ")
# print("- " *30)
# print(population.iloc[:4, :4])
# print("- " *30)

# print(" annual data sample ")
# print("- " *30)
# print(annual.iloc[:4, :4])
# print("- " *30)

# print(" land data sample ")
# print("- " *30)
# print(land.iloc[:4, :4])
# print("- " *30)

  BUYER_COUNTY BUYER_STATE STATE COUNTY  year population
0      AUTAUGA          AL     1      1  2006      51328
1      BALDWIN          AL     1      3  2006     168121
2      BARBOUR          AL     1      5  2006      27861
3         BIBB          AL     1      7  2006      22099
4       BLOUNT          AL     1      9  2006      55485
